In [1]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.models import resnet18

In [2]:
from tensorboardX import SummaryWriter

EXPERIMENT = 'resnet18'

datdir = os.path.join(os.getcwd(), 'Skin40')
print(datdir)

/home/jovyan/SYSUDeepLearningSkin/Skin40


In [3]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,transform=None):
        self.data_path = []
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((784,784)),
                transforms.RandomRotation(5),
                transforms.ToTensor()
            ])
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
#                 img = Image.open(fil)
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        result = (self.transform(img) , label)
        del img
        return result
    
    def __len__(self):
        return len(self.data_path)

In [4]:
ds = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40,training=True)

In [5]:
print(len(ds))
print(ds[100][0].sum())

1920
tensor(1048541.2500)


In [6]:
# del training_dataloader
training_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40)
training_dataloader = DataLoader(training_dataset , batch_size = 3 , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = 3 , num_workers = 1, shuffle = True)

In [7]:
def train(model, train_loader, loss_func, optimizer, device):
    """
    train model using loss_fn and optimizer in an epoch.
    model: CNN networks
    train_loader: a Dataloader object with training data
    loss_func: loss function
    device: train on cpu or gpu device
    """
    total_loss = 0
    # train the model using minibatch
    
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        
        # forward
        outputs = model(images)
        loss = loss_func(outputs, targets)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

In [8]:
def evaluate(model, val_loader, device):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        
        # record all target && correct prediction by each class
        all_tar = np.zeros((40)).tolist()
        cor_pre =  np.zeros((40)).tolist()

        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            
            # return the maximum value of each row of the input tensor in the 
            # given dimension dim, the second return vale is the index location
            # of each maxium value found(argmax)
            _, predicted = torch.max(outputs.data, dim=1)
            cor_bol_list = (predicted == targets)
            correct += (cor_bol_list).sum().item()
            total += targets.size(0)
            # 
            for i,bl in enumerate( cor_bol_list):
                all_tar[targets[i].cpu().numpy().tolist()] += 1
                if bl:
                    cor_pre[predicted[i].cpu().numpy().tolist()] += 1
        # accuracy of tatal && each class 
        accuracy = correct / total
        each_acc = torch.tensor(cor_pre) / torch.tensor(all_tar)
        return accuracy,each_acc

In [9]:
def fit(model, num_epochs, optimizer, device):
    """
     train and evaluate an classifier num_epochs times.
    n and evaluate an classifier num_epochs times.
    We use optimizer and cross entropy loss to train the model. 
    Args: 
        model: CNN network
        num_epochs: the number of training epochs
        optimizer: optimize the loss function    loss_func.to(device)
    loss_func.to(device)

    """
    # loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    
    model.to(device)
    loss_func.to(device)
    
    # log train loss and test accuracy
    losses = []
    accs = []
    writer = SummaryWriter(log_dir='log/%s training loss' % EXPERIMENT)
    
    for epoch in range(num_epochs):
        print(epoch)        
        # tensorboard
        # train step
        loss = train(model, training_dataloader, loss_func, optimizer, device)
        
        # save model
        torch.save(model.state_dict(), 'model_%s.pth' % EXPERIMENT)
        
        accuracy,each_acc = evaluate(model, training_dataloader, device)   
        print('train')
        writer.add_scalar('training loss', loss, epoch)
        writer.add_scalar('training acc', accuracy, epoch)        
        for i in range(40):
            temp_writer = SummaryWriter(log_dir='log/%s' % EXPERIMENT + ' training class %s acc' % str(i))
            temp_writer.add_scalar('training class acc',each_acc[i], epoch)
            temp_writer.close()
            
        # evaluate step
        accuracy,each_acc = evaluate(model, test_dataloader, device) 
        print('eval')
        writer.add_scalar('testing acc', accuracy, epoch)
        for i in range(40):
            temp_writer = SummaryWriter(log_dir='log/%s' % EXPERIMENT + 'testing class %s acc' % str(i))
            writer.add_scalar('testing class acc',each_acc[i], epoch)
            temp_writer.close()
    writer.close()


In [10]:
model = resnet18(pretrained=True)

In [11]:
# Hyper-parameters
num_epochs = 2
lr = 0.001
# Device configuration
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

print(device)
#for parma in model.parameters():
#    parma.requires_grad = False
'''
model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 40))
'''
model.fc = torch.nn.Sequential(nn.Linear(512, 40))
cost = torch.nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)

fit(model, num_epochs, optimizer, device)  #

cuda:1
0
train
eval
1
train
eval


In [12]:
for i,(dat, tar) in enumerate(test_dataloader):
    print(i ,dat.shape, tar)
    if ( i>30):
        break

0 torch.Size([3, 3, 784, 784]) tensor([ 4, 32, 37])
1 torch.Size([3, 3, 784, 784]) tensor([ 2, 37, 33])
2 torch.Size([3, 3, 784, 784]) tensor([15, 14, 17])
3 torch.Size([3, 3, 784, 784]) tensor([13, 25, 23])
4 torch.Size([3, 3, 784, 784]) tensor([37, 23, 25])
5 torch.Size([3, 3, 784, 784]) tensor([22,  5,  7])
6 torch.Size([3, 3, 784, 784]) tensor([35, 34, 10])
7 torch.Size([3, 3, 784, 784]) tensor([19, 14, 14])
8 torch.Size([3, 3, 784, 784]) tensor([15, 36, 11])
9 torch.Size([3, 3, 784, 784]) tensor([ 0, 28, 25])
10 torch.Size([3, 3, 784, 784]) tensor([20, 10, 39])
11 torch.Size([3, 3, 784, 784]) tensor([ 2, 33, 16])
12 torch.Size([3, 3, 784, 784]) tensor([38, 32, 24])
13 torch.Size([3, 3, 784, 784]) tensor([37, 23,  2])
14 torch.Size([3, 3, 784, 784]) tensor([36, 32,  9])
15 torch.Size([3, 3, 784, 784]) tensor([10,  6, 13])
16 torch.Size([3, 3, 784, 784]) tensor([24, 31, 26])
17 torch.Size([3, 3, 784, 784]) tensor([22, 23, 30])
18 torch.Size([3, 3, 784, 784]) tensor([28,  2, 33])
19 